In [3]:
import gradio as gr
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import datetime

# Define option pricing models
def black_scholes(S, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

def greeks(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Delta
    delta_call = norm.cdf(d1)
    delta_put = delta_call - 1

    # Gamma
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))

    # Theta
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2))
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2))

    # Vega
    vega = S * norm.pdf(d1) * np.sqrt(T)

    # Rho
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return {
        "delta_call": delta_call, 
        "delta_put": delta_put, 
        "gamma": gamma, 
        "theta_call": theta_call, 
        "theta_put": theta_put, 
        "vega": vega, 
        "rho_call": rho_call, 
        "rho_put": rho_put
    }

def monte_carlo(S, K, T, r, sigma, option_type="call", num_simulations=10000):
    dt = T / 252
    option_price = 0
    for _ in range(num_simulations):
        ST = S * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * np.random.normal())
        if option_type == "call":
            option_price += max(0, ST - K)
        elif option_type == "put":
            option_price += max(0, K - ST)
    return (option_price / num_simulations) * np.exp(-r * T)

def binomial_tree(S, K, T, r, sigma, steps, option_type="call", american=True):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    option_values = np.zeros((steps + 1, steps + 1))

    for i in range(steps + 1):
        ST = S * (u ** i) * (d ** (steps - i))
        if option_type == "call":
            option_values[i, steps] = max(0, ST - K)
        elif option_type == "put":
            option_values[i, steps] = max(0, K - ST)

    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_values[i, j] = np.exp(-r * dt) * (p * option_values[i + 1, j + 1] + (1 - p) * option_values[i, j + 1])
            if american:
                if option_type == "call":
                    option_values[i, j] = max(option_values[i, j], S * (u ** i) * (d ** (j - i)) - K)
                elif option_type == "put":
                    option_values[i, j] = max(option_values[i, j], K - S * (u ** i) * (d ** (j - i)))

    return option_values[0, 0]

def get_historical_data(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        data['Returns'] = data['Adj Close'].pct_change()
        sigma = np.std(data['Returns']) * np.sqrt(252)  # Annualized volatility
        return data, sigma
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None, None

def plot_adjusted_close(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data['Adj Close'])
    plt.title('Adjusted Close Price Over Time')
    plt.xlabel('Date')
    plt.ylabel('Adjusted Close Price')
    plt.grid(True)
    plt.tight_layout()
    return plt

def option_pricing_interface(ticker, start_date, end_date, S, K, r, sigma, steps, model, option_style, option_type, calculate_greeks):
    data, calculated_sigma = get_historical_data(ticker, start_date, end_date)
    if data is None:
        return "Failed to fetch historical data.", "", None, ""

    T = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365

    # Ensure all inputs are valid
    if S <= 0 or K <= 0 or T <= 0 or r < 0 or sigma < 0:
        return "Invalid inputs provided. Ensure all values are positive and non-zero.", "", plot_adjusted_close(data), ""

    # If sigma is None or not provided, use the calculated sigma
    if sigma == 0:
        sigma = calculated_sigma

    # Handle unsupported option styles for Black-Scholes and Monte Carlo
    if model == "Black-Scholes" and option_style == "American":
        return "Black-Scholes model does not support American options", "Black-Scholes model does not support American options", plot_adjusted_close(data), ""
    elif model == "Monte Carlo" and option_style == "American":
        return "Monte Carlo model does not support American options", "Monte Carlo model does not support American options", plot_adjusted_close(data), ""

    # Calculate option prices based on selected model
    if model == "Black-Scholes":
        call_price = black_scholes(S, K, T, r, sigma, "call")
        put_price = black_scholes(S, K, T, r, sigma, "put")
        if calculate_greeks:
            greeks_result = greeks(S, K, T, r, sigma)  # Calculate Greeks if selected
            greeks_output = f"""
            **Greeks for Call Option**  
            - Delta: {greeks_result['delta_call']:.4f}  
            - Gamma: {greeks_result['gamma']:.4f}  
            - Theta: {greeks_result['theta_call']:.4f}  
            - Vega: {greeks_result['vega']:.4f}  
            - Rho: {greeks_result['rho_call']:.4f}  

            **Greeks for Put Option**  
            - Delta: {greeks_result['delta_put']:.4f}  
            - Gamma: {greeks_result['gamma']:.4f}  
            - Theta: {greeks_result['theta_put']:.4f}  
            - Vega: {greeks_result['vega']:.4f}  
            - Rho: {greeks_result['rho_put']:.4f}  
            """
        else:
            greeks_output = "Greeks calculation not selected"
    elif model == "Monte Carlo":
        call_price = monte_carlo(S, K, T, r, sigma, "call")
        put_price = monte_carlo(S, K, T, r, sigma, "put")
        greeks_output = "Greeks not calculated for Monte Carlo"
    elif model == "Binomial":
        if option_style == "European":
            call_price = binomial_tree(S, K, T, r, sigma, steps, "call", american=False)
            put_price = binomial_tree(S, K, T, r, sigma, steps, "put", american=False)
        else:
            call_price = binomial_tree(S, K, T, r, sigma, steps, "call", american=True)
            put_price = binomial_tree(S, K, T, r, sigma, steps, "put", american=True)
        greeks_output = "Greeks not calculated for Binomial"

    # Generate and return the adjusted close price graph
    graph = plot_adjusted_close(data)

    return call_price, put_price, graph, greeks_output

# Gradio interface setup with GUI enhancements and Greeks calculation option
with gr.Blocks() as interface:
    gr.Markdown("# Option Pricing Calculator")
    
    with gr.Row():
        ticker = gr.Textbox(label="Stock Ticker", placeholder="AAPL")
        start_date = gr.Textbox(label="Start Date (YYYY-MM-DD)", placeholder="2023-01-01")
        end_date = gr.Textbox(label="End Date (YYYY-MM-DD)", placeholder="2023-12-31")
    
    with gr.Row():
        S = gr.Number(label="Spot Price", value=150)
        K = gr.Number(label="Strike Price", value=160)
    
    with gr.Row():
        r = gr.Slider(0.0, 0.1, step=0.001, label="Risk-Free Rate")
        sigma = gr.Slider(0.0, 1.0, step=0.01, label="Sigma (Volatility)")
        steps = gr.Slider(1, 100, step=1, label="Number of Steps")
    
    with gr.Row():
        model = gr.Dropdown(["Black-Scholes", "Monte Carlo", "Binomial"], label="Model")
        option_style = gr.Dropdown(["European", "American"], label="Option Style")
        option_type = gr.Dropdown(["call", "put"], label="Option Type")
        calculate_greeks = gr.Checkbox(label="Calculate Greeks?", value=False)  # New checkbox for Greeks calculation
    
    call_output = gr.Textbox(label="Call Option Price or Message")
    put_output = gr.Textbox(label="Put Option Price or Message")
    graph_output = gr.Plot(label="Adjusted Close Price Graph")
    greeks_output = gr.Markdown(label="Greeks")  # Use Markdown for formatting the Greeks output

    # Define button with actions
    calculate_button = gr.Button("Calculate Option Prices")
    calculate_button.click(option_pricing_interface, 
                           inputs=[ticker, start_date, end_date, S, K, r, sigma, steps, model, option_style, option_type, calculate_greeks], 
                           outputs=[call_output, put_output, graph_output, greeks_output])

interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7879

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/09/03 19:19:45 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
